# Data Cleaning 3: Outliers

### Imports and getting set up

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

### DB Connection & Call

In [2]:
confile = list(pd.read_csv('../../dbcon.csv'))
postgres_db = 'useducation'
db_connection = 'postgresql://{}:{}@{}:{}/{}'.format(confile[0], confile[1], confile[2], confile[3], postgres_db)

In [3]:
query = '''
SELECT *
FROM useducation
;'''

useducation = pd.read_sql(query, db_connection)
useducation.sort_values('PRIMARY_KEY').reset_index(drop=True).head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_AMERICAN_SAMOA,AMERICAN_SAMOA,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,981.0,886.0,748.0,7969.0,3355.0,12786.0,NaN,NaN,NaN,NaN
3,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
4,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665


### Handling nulls

In [4]:
display(
    useducation.isna().mean().sort_values(ascending=False)
)

AVG_READING_8_SCORE             0.666220
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_MATH_4_SCORE                0.640751
OTHER_EXPENDITURE               0.176273
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_ALL_G                    0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
YEAR                            0.000000
STATE                           0.000000
PRIMARY_KEY     

In [5]:
fillcols = [column for column in useducation.columns if useducation[column].isna().sum() > 0]
for col in fillcols:
    useducation.loc[:, col] = useducation.loc[:, col].fillna(useducation.loc[:, col].mean())

In [6]:
display(
    useducation.isna().mean().sort_values(ascending=False)
)

AVG_READING_8_SCORE             0.0
OTHER_EXPENDITURE               0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
AVG_READING_4_SCORE             0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
PRIMARY_KEY                     0.0
dtype: float64

#### 1. Consider the two variables: _TOTAL_REVENUE_ and _TOTAL_EXPENDITURE_. Do these variables have outlier values?

In [7]:
q75, q25 = np.percentile(useducation['TOTAL_REVENUE'], [80,20])
iqr = q75 - q25

min_val = q25 - (iqr * 1.5)
max_val = q75 + (iqr * 1.5)
display(
    len(
        np.where(
            (useducation['TOTAL_REVENUE'] > max_val) | 
            (useducation['TOTAL_REVENUE'] < min_val)
        )[0]
    )
)


167

In [8]:
q75, q25 = np.percentile(useducation['TOTAL_EXPENDITURE'], [80,20])
iqr = q75 - q25

min_val = q25 - (iqr * 1.5)
max_val = q75 + (iqr * 1.5)
display(
    len(
        np.where(
            (useducation['TOTAL_EXPENDITURE'] > max_val) | 
            (useducation['TOTAL_EXPENDITURE'] < min_val)
        )[0]
    )
)


105

#### 2. If you detect outliers in the _TOTAL_REVENUE_ and _TOTAL_EXPENDITURE_ variables, apply the techniques you learned in this checkpoint to eliminate them and validate that there's no outlier values after you handled them.

In [9]:
useducation2 = useducation.copy()
useducation2['TOTAL_REVENUE'] = st.mstats.winsorize(useducation['TOTAL_REVENUE'], (0, 0.1))
useducation2['TOTAL_EXPENDITURE'] = st.mstats.winsorize(useducation['TOTAL_EXPENDITURE'], (0, 0.1))

In [10]:
display(
    len(
        np.where(
            (useducation2['TOTAL_REVENUE'] > max_val) | 
            (useducation2['TOTAL_REVENUE'] < min_val)
        )[0]
    ),
    len(
        np.where(
            (useducation2['TOTAL_EXPENDITURE'] > max_val) | 
            (useducation2['TOTAL_EXPENDITURE'] < min_val)
        )[0]
    )
)

0

0

#### 3. Create another variable by subtracting the original _TOTAL_EXPENDITURE_ from _TOTAL_REVENUE_ (before you eliminated the outliers). You can think of it as a kind of budget deficit in education. Do you find any outlier values in this new variable? If so, eliminate them using the technique you think most suitable.

In [27]:
useducation3 = useducation.copy()
useducation3['DEFICIT'] = useducation3['TOTAL_REVENUE'] - useducation3['TOTAL_EXPENDITURE']

q75, q25 = np.percentile(useducation3['DEFICIT'], [80,20])
iqr = q75 - q25

min_val = q25 - (iqr * 1.5)
max_val = q75 + (iqr * 1.5)

In [28]:
display(
    len(
        np.where(
            (useducation3['DEFICIT'] > max_val) | 
            (useducation3['DEFICIT'] < min_val)
        )[0]
    )
)

163

In [29]:
useducation3['DEFICIT'] = st.mstats.winsorize(useducation3['DEFICIT'], (0, 0.1))

In [30]:
display(
    len(
        np.where(
            (useducation3['DEFICIT'] > max_val) | 
            (useducation3['DEFICIT'] < min_val)
        )[0]
    )
)

97

#### 4. Now create another variable by subtracting the TOTAL_EXPENDITURE from TOTAL_REVENUE. This time, use the outlier eliminated versions of TOTAL_EXPENDITURE from TOTAL_REVENUE. In this newly created variable, can you find any outliers? If so, eliminate them.

In [31]:
useducation4 = useducation.copy()
useducation4['TOTAL_REVENUE'] = st.mstats.winsorize(useducation4['TOTAL_REVENUE'], (0, 0.1))
useducation4['TOTAL_EXPENDITURE'] = st.mstats.winsorize(useducation4['TOTAL_EXPENDITURE'], (0, 0.1))
useducation4['DEFICIT'] = useducation4['TOTAL_REVENUE'] - useducation4['TOTAL_EXPENDITURE']
display(
    len(
        np.where(
            (useducation4['DEFICIT'] > max_val) | 
            (useducation4['DEFICIT'] < min_val)
        )[0]
    )
)

72

#### 5. Compare some basic descriptive statistics of the budget variables you end up with in the 3rd and the 4th questions. Do you see any differences?

In [32]:
display(
    useducation3.describe(),
    useducation4.describe()
)

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,DEFICIT
count,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,...,1492.000000,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1492.000000,1492.000000,1492.000000,1492.000000,1.492000e+03
mean,2004.433646,9.159308e+05,9.092082e+06,7.663723e+05,4.216553e+06,4.109157e+06,9.196681e+06,4.762966e+06,2.680331e+06,4.292046e+05,...,64271.057311,54268.924320,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132,-1.425569e+05
std,7.393983,9.667725e+05,1.087818e+07,1.060702e+06,5.133895e+06,5.078230e+06,1.109393e+07,5.828468e+06,3.105812e+06,4.846832e+05,...,75364.301721,64838.328605,6.147133e+05,2.938773e+05,9.126522e+05,6.122843,6.063038,4.641053,3.928132,4.671753e+05
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,437.000000,311.000000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102,-5.487742e+06
25%,1998.000000,3.150940e+05,2.546261e+06,2.170200e+05,1.356434e+06,9.447962e+05,2.523968e+06,1.343611e+06,7.673688e+05,1.354282e+05,...,15682.750000,13437.000000,1.211522e+05,5.919750e+04,2.293578e+05,234.768293,278.414711,218.866154,263.661132,-1.360070e+05
50%,2004.000000,8.204140e+05,6.359310e+06,5.167410e+05,3.127639e+06,2.697257e+06,6.520224e+06,3.358142e+06,1.913844e+06,3.672335e+05,...,49571.000000,39566.000000,4.015865e+05,1.817010e+05,6.473260e+05,234.768293,278.414711,218.866154,263.661132,-4.849350e+04
75%,2011.000000,9.211780e+05,9.276958e+06,7.663723e+05,4.269811e+06,4.109157e+06,9.434922e+06,4.938820e+06,2.795888e+06,4.568915e+05,...,72131.000000,64616.250000,5.749118e+05,2.825390e+05,8.724220e+05,234.768293,278.414711,218.866154,263.661132,2.965425e+04
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,500143.000000,498403.000000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130,2.085280e+05


,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,DEFICIT
count,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,...,1492.000000,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1492.000000,1492.000000,1492.000000,1492.000000,1.492000e+03
mean,2004.433646,9.159308e+05,7.518248e+06,7.663723e+05,4.216553e+06,4.109157e+06,7.582929e+06,4.762966e+06,2.680331e+06,4.292046e+05,...,64271.057311,54268.924320,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132,-6.468178e+04
std,7.393983,9.667725e+05,5.723655e+06,1.060702e+06,5.133895e+06,5.078230e+06,5.785973e+06,5.828468e+06,3.105812e+06,4.846832e+05,...,75364.301721,64838.328605,6.147133e+05,2.938773e+05,9.126522e+05,6.122843,6.063038,4.641053,3.928132,2.682253e+05
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,437.000000,311.000000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102,-1.531644e+06
25%,1998.000000,3.150940e+05,2.546261e+06,2.170200e+05,1.356434e+06,9.447962e+05,2.523968e+06,1.343611e+06,7.673688e+05,1.354282e+05,...,15682.750000,13437.000000,1.211522e+05,5.919750e+04,2.293578e+05,234.768293,278.414711,218.866154,263.661132,-1.811838e+05
50%,2004.000000,8.204140e+05,6.359310e+06,5.167410e+05,3.127639e+06,2.697257e+06,6.520224e+06,3.358142e+06,1.913844e+06,3.672335e+05,...,49571.000000,39566.000000,4.015865e+05,1.817010e+05,6.473260e+05,234.768293,278.414711,218.866154,263.661132,-6.565100e+04
75%,2011.000000,9.211780e+05,9.276958e+06,7.663723e+05,4.269811e+06,4.109157e+06,9.434922e+06,4.938820e+06,2.795888e+06,4.568915e+05,...,72131.000000,64616.250000,5.749118e+05,2.825390e+05,8.724220e+05,234.768293,278.414711,218.866154,263.661132,2.047350e+04
max,2017.000000,6.307022e+06,1.937403e+07,9.990221e+06,5.090457e+07,3.610526e+07,1.968037e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,500143.000000,498403.000000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130,2.520706e+06


#### 6. If our variable of interest is the budget deficit variable, which method do you think is the appropriate in dealing with the outliers in this variable: the method in the 3rd question or the one in the 4th question?

<span style="color:blue">The third method since you want to touch the raw data as little as possible until you get the feature you wish to engineer calculated out.</span>